In [43]:
import onnxruntime
onnxruntime.get_device()
import torch
from torch import nn
import torch.nn.functional as F

In [44]:
class Cnn(nn.Module):
    def __init__(self, input_shape):
        super(Cnn, self).__init__()
        self.input_shape = input_shape

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape[0], out_channels=32, kernel_size=3, padding=0, stride = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(18496, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x.squeeze(1)

input_shape = (3, 150, 150)

from torchsummary import summary
#summary(model, input_size=input_shape)

In [45]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1234)
if device =='cuda':
    torch.cuda.manual_seed_all(1234)

In [46]:
model = torch.load('cnn-torch-100.pth')
model.eval()
x = torch.randn(1,3, 150, 150).to(device)
with torch.no_grad():
    torch.onnx.export(
        model,                       # 要转换的模型
        x,                           # 模型的任意一组输入
        'cnn-torch-100.onnx',    # 导出的 ONNX 文件名
        opset_version=11,            # ONNX 算子集版本
        input_names=['input'],       # 输入 Tensor 的名称（自己起名字）
        output_names=['output']      # 输出 Tensor 的名称（自己起名字）
    )

In [47]:
ort_session = onnxruntime.InferenceSession('cnn-torch-100.onnx')

In [48]:
device = 'cuda'
input_data = torch.zeros(1,3,150,150) # 创建一个形状为(1, 18496)的张量
for i in range(0,3):
    for j in range (0,150):
        for k in range (0,150):
            input_data[0,i,j,k] = torch.sin(torch.tensor(i*150*150+j*150+k)) # 使用PyTorch的sin函数

In [49]:
ort_inputs = {'input': input_data.numpy()}
pred= ort_session.run(['output'], ort_inputs)[0]
print(pred)


[1.]
